# Containers

samples-eda\samples\containers.csv

```csv
zone,container_id,width_cm,depth_cm,height_cm
Sanitation_Bay,SB01,25.0,42.5,200.0
Sanitation_Bay,SB02,100.0,85.0,200.0
Sanitation_Bay,SB03,100.0,85.0,200.0
Sanitation_Bay,SB04,200.0,85.0,200.0
Command_Center,CC01,100.0,85.0,200.0
```

# Items

samples-eda\samples\input_items.csv

```csv
item_id,name,width_cm,depth_cm,height_cm,mass_kg,priority,expiry_date,usage_limit,preferred_zone
000001,Research_Samples,26.8,17.5,19.4,2.4,84,N/A,2304,Storage_Bay
000002,LED_Work_Light,49.9,36.3,44.2,40.03,90,N/A,3558,Maintenance_Bay
000003,Pressure_Regulator,48.1,33.2,43.1,34.41,16,N/A,1075,Airlock
000004,Emergency_Oxygen_Mask,15.6,46.5,17.0,6.17,42,N/A,709,Medical_Bay
000005,Battery_Pack,23.2,31.6,18.2,6.67,93,N/A,175,External_Storage
```

JSON Shape

Container:

```json
{
  "zone": "Sanitation_Bay",
  "container_id": "SB01",
  "width_cm": 25.0,
  "depth_cm": 42.5,
  "height_cm": 200.0
}
```

Item:

```json
{
  "item_id": "000001",
  "name": "Research_Samples",
  "width_cm": 26.8,
  "depth_cm": 17.5,
  "height_cm": 19.4,
  "mass_kg": 2.4,
  "priority": 84,
  "expiry_date": "N/A",
  "usage_limit": 2304,
  "preferred_zone": "Storage_Bay"
}
```

py3dbp Classes

```python
class Bin(
    name: str,
    width: float,
    height: float,
    depth: float,
    max_weight: int
)

class Item(
    name: str,
    width: float,
    height: float,
    depth: float,
    weight: int
)
```

In [3]:
import json
from py3dbp import Packer, Bin, Item
import time


def initialize_packer():
  packer = Packer()
  packer.add_bin(Bin('container', 100, 100, 100, 1000))
  return packer


def load_items(packer, file_path):
  with open(file_path, "r") as f:
    data = json.load(f)
    items = []
    for i, entry in enumerate(data["items"], 1):
      size = entry["size"]
      item = Item(f"item{i}", size["width"], size["height"], size["depth"], 0.01)
      items.append(item)
    items.sort(key=lambda x: x.get_volume())  # Sort items by volume, has no effect on the result
    for item in items:
      packer.add_item(item)


def pack_items(packer):
  packer.pack(distribute_items=True)


def generate_output_json(packer, output_path):
  result = {"items": []}
  for b in packer.bins:
    for item in b.items:
      dim = item.get_dimension()
      pos = item.position
      result["items"].append({
          "size": {"width": float(dim[0]), "height": float(dim[1]), "depth": float(dim[2])},
          "position": {"x": float(pos[0]), "y": float(pos[1]), "z": float(-pos[2])}
      })
  with open(output_path, "w") as f:
    json.dump(result, f, indent=2)


def print_metrics(metrics):
  total_volume, fitted_volume, fitted_percentage, unfitted_volume, unfitted_percentage, fitted_items, unfitted_items = metrics

  print(f"Total Volume: {total_volume}")
  print(f"☑️ Fitted Items: {fitted_items}")
  print(f"☑️ Fitted Volume: {fitted_volume}")
  print(f"☑️ Fitted Percentage: {fitted_percentage:.2f}%")

  print(f"Unfitted Items: {unfitted_items}")
  print(f"Unfitted Volume: {unfitted_volume}")
  print(f"Unfitted Percentage: {unfitted_percentage:.2f}%")


def calculate_metrics(packer):
  total_volume = 100 * 100 * 100
  fitted_volume = 0
  unfitted_volume = 0
  fitted_items = 0
  unfitted_items = 0

  for b in packer.bins:
    for item in b.items:
      fitted_volume += item.get_volume()
      fitted_items += 1
    for item in b.unfitted_items:
      unfitted_volume += item.get_volume()
      unfitted_items += 1

  fitted_percentage = (fitted_volume / total_volume) * 100
  unfitted_percentage = (unfitted_volume / total_volume) * 100

  return total_volume, fitted_volume, fitted_percentage, unfitted_volume, unfitted_percentage, fitted_items, unfitted_items


def main(dataset_name):
  input_path = rf"C:\k26rahul\Code\space-hackathon\3d-visualizer\data\{dataset_name}.json"
  output_path = r"C:\k26rahul\Code\space-hackathon\3d-visualizer\data\output.json"

  start_time = time.time()

  packer = initialize_packer()
  load_items(packer, input_path)
  pack_items(packer)
  generate_output_json(packer, output_path)
  metrics = calculate_metrics(packer)
  print_metrics(metrics)

  end_time = time.time()
  print(f"\nExecution time: {end_time - start_time:.3f} seconds")


main(dataset_name='17mixed')

Total Volume: 1000000
☑️ Fitted Items: 6
☑️ Fitted Volume: 150000.000
☑️ Fitted Percentage: 15.00%
Unfitted Items: 1
Unfitted Volume: 850000.000
Unfitted Percentage: 85.00%

Execution time: 0.002 seconds


In [16]:
def generate_cube_set(size: int):
  result = {"items": []}
  container_size = 100
  items_per_axis = container_size // size
  total_items = items_per_axis ** 3

  name = f"{total_items}cube{size}"
  output_path = rf"C:\k26rahul\Code\space-hackathon\3d-visualizer\data\{name}.json"

  for x in range(items_per_axis):
    for y in range(items_per_axis):
      for z in range(items_per_axis):
        result["items"].append({
            "size": {"width": size, "height": size, "depth": size},
            "position": {"x": x * size, "y": y * size, "z": -z * size}
        })

  print(f"Generated {len(result['items'])} items as '{name}'")
  with open(output_path, "w") as f:
    json.dump(result, f, indent=2)


generate_cube_set(20)
generate_cube_set(10)
generate_cube_set(5)
generate_cube_set(4)

Generated 125 items as '125cube20'
Generated 1000 items as '1000cube10'
Generated 8000 items as '8000cube5'
Generated 15625 items as '15625cube4'


In [24]:
def generate_cuboid_set(width: int, height: int, depth: int):
  result = {"items": []}
  container_size = 100
  x_count = container_size // width
  y_count = container_size // height
  z_count = container_size // depth
  total_items = x_count * y_count * z_count

  name = f"{total_items}-{width}x{height}x{depth}"
  output_path = rf"C:\k26rahul\Code\space-hackathon\3d-visualizer\data\{name}.json"

  for x in range(x_count):
    for y in range(y_count):
      for z in range(z_count):
        result["items"].append({
            "size": {"width": width, "height": height, "depth": depth},
            "position": {"x": x * width, "y": y * height, "z": -z * depth}
        })

  print(f"Generated {len(result['items'])} items as '{name}'")
  with open(output_path, "w") as f:
    json.dump(result, f, indent=2)


generate_cuboid_set(30, 20, 10)
generate_cuboid_set(10, 40, 20)
generate_cuboid_set(30, 5, 4)
generate_cuboid_set(10, 5, 4)
generate_cuboid_set(100, 20, 10)
generate_cuboid_set(80, 10, 10)

Generated 150 items as '150-30x20x10'
Generated 100 items as '100-10x40x20'
Generated 1500 items as '1500-30x5x4'
Generated 5000 items as '5000-10x5x4'
Generated 50 items as '50-100x20x10'
Generated 100 items as '100-80x10x10'


In [25]:
import json
import os
import glob

# Directory containing JSON files
directory = r"C:\k26rahul\Code\space-hackathon\3d-visualizer\data"

# Default container with ID
default_container = {
    "id": "CONTAINER_001",
    "size": {"width": 100, "height": 100, "depth": 100},
    "position": {"x": 0, "y": 0, "z": 0}
}


def update_json_file(file_path):
  try:
    # Read existing JSON
    with open(file_path, 'r') as f:
      data = json.load(f)

    if isinstance(data, dict) and "items" in data:
      # Add containers array
      data["containers"] = [default_container]

      # Update all items to include container_id
      for item in data["items"]:
        item["container_id"] = default_container["id"]

      # Write back to file
      with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)

      print(f"Updated: {os.path.basename(file_path)}")
  except Exception as e:
    print(f"Error processing {os.path.basename(file_path)}: {str(e)}")


# Process all JSON files in the directory
json_files = glob.glob(os.path.join(directory, "*.json"))
for file_path in json_files:
  update_json_file(file_path)

print("\nFinished processing all JSON files")

Updated: 100-10x40x20.json
Updated: 100-80x10x10.json
Updated: 1000-10x10x10.json
Updated: 1000cube10.json
Updated: 125cube20.json
Updated: 150-30x20x10.json
Updated: 1500-30x5x4.json
Updated: 15625cube4.json
Updated: 17mixed.json
Updated: 17mixed2.json
Updated: 31mixed.json
Updated: 40-10x40x50.json
Updated: 400-10x40x5.json
Updated: 50-100x20x10.json
Updated: 500-10x20x10.json
Updated: 5000-10x5x4.json
Updated: 65stairs.json
Updated: 7giant.json
Updated: 8000cube5.json
Updated: 8cube50.json
Updated: output.json

Finished processing all JSON files
